# ---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

In [4]:
def date_sorter():
    
    r1 = '((?:\d{,2}\s)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:-|\.|\s|,)\s?\d{,2}[a-z]*(?:-|,|\s)?\s?\d{2,4})'
    s1 = df.str.extract(r1)

    r2 = '((?:\d{1,2})(?:(?:\/|-)\d{1,2})(?:(?:\/|-)\d{2,4}))'
    s2 = df.str.extract(r2)
    
    r3 = '((?:\d{1,2}(?:-|\/))?\d{4})'
    s3 = df.str.extract(r3)
    
    final = s1.fillna(s2).fillna(s3) 
    
    print(final.columns)
    final.columns=['date']
    final['date'] = final['date'].apply(lambda x: str(x))
    final['date'] = final['date'].str.strip()
    
    month_dict = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, \
                  'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12, 'January':1, \
                  'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, \
                  'August':8, 'September':9, 'October':10, 'November':11, \
                  'December': 12, 'Decemeber':12, 'Janaury':1}
             
    final['date']  = final['date'].str.replace('-','/')
    final['date']  = final['date'].str.replace(' ','/')
    final['date']  = final['date'].str.replace('.','')
    final['date']  = final['date'].str.replace(',','')
        
    new = final['date'].str.split('/',expand=True)
    final['day'] = new[0]
    final['month'] = new[1]
    final['year'] = new[2]
    
    final['day'] = final['day'].replace('Decemeber','December',regex=True).replace('Janaury','January',regex=True)
    
    # swap :125 month and day column values
    new = final['month'][:125].copy()
    final['month'][:125] = final['day'][:125]
    final['day'][:125] = new
    
    # swap 194:228 month and day column values
    new = final['month'][194:228].copy()
    final['month'][194:228] = final['day'][194:228]
    final['day'][194:228] = new

    # 228:455
    final['year'][228:455] = final['month'][228:455].copy()
    final['month'][228:455] = final['day'][228:455].copy()
    
    final['day'][228:455] = pd.Series([1 for i in range(final['month'][228:455].shape[0])]).copy()
    
    final['month'][125:343] = final['month'][125:343].apply(lambda x: month_dict[x])
    
    final['year'][455:] = final['day'][455:].copy()
    final['month'][455:] = pd.Series([1 for i in range(final['year'][455:].shape[0])]).copy()
    final['day'][455:] = pd.Series([1 for i in range(final['year'][455:].shape[0])]).copy()
    
    for index, val in final['year'].iteritems():
        if len(val) == 2:
            val = '19'+val
    
#     final['year'] = final['year'].apply(mapfn)

    final['day'] = final['day'].apply(lambda x: str(x))
    final['month'] = final['month'].apply(lambda x: str(x))
    final['year'] = final['year'].apply(lambda x: str(x))
    
    final['new date'] = final['month'] + '/' + final['day'] +  '/' + final['year']
    final['new date'] = pd.to_datetime(final['new date'])
    
    final.sort_values(by='new date', inplace=True)
    index_series = pd.Series(list(final.index))
    
    return index_series


In [5]:
date_sorter()

Int64Index([0], dtype='int64')


0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    427
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64